## Data Acquisition and Consolidation

In [510]:
#Importing Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import glob

## Data Sources

### Datasets

1. [Unemployment Claims](https://apps.deed.state.mn.us/lmi/ui/Results.aspx?area=R11000&date=202209)
3. [Census Data](https://qwiexplorer.ces.census.gov/static/explore.html#x=0&g=0) 
6. [Covid Transmission](https://catalog.data.gov/dataset/united-states-covid-19-county-level-of-community-transmission-historical-changes)
5. [Daily new Cases (Prev)](https://www.kaggle.com/datasets/antgoldbloom/covid19-data-from-john-hopkins-university)
6. [Mask Policy (Prev)](https://data.cdc.gov/Policy-Surveillance/U-S-State-and-Territorial-Public-Mask-Mandates-Fro/62d6-pm5i)

### Data Shape

1. Granularity  :    Monthly (Time). All data is rolled up to the monthly level. This was handled differently for different datasets and is described during the consolidation process. <br><br>
2. Region       :    All the data is for the Economic Region R11000 , has the following counties :
- Anoka County  <br>
- Carver County	 <br>
- Dakota County	 <br>
- Hennepin County	 <br>
- Ramsey County	 <br>
- Scott County		 <br>
- Washington Count <br><br>

## Data Acquisition

### 1. Unemployment Claims Data


In [511]:
def get_claims(folder,str): #name of sub-folder and the 
    """
    Constructs dataframe by concatenating all the csv's from a specified folder for demographics and industries. 

    Parameters
    ----------
        folder : str
            name of the folder with the needed csv files
        str : str
            name of the column for the joined category (eg 'sex' for male and female categories)

    Returns
    -------
        df : pandas Datafram
            Final dataframe schema with required values for analysis 
            Columns:
                Name: year, dtype: int64
                Name: month, dtype: int64
                Name: {str}, dtype: object
                Name: claims, dtype: float64
    """
    files = glob.glob(folder+"/*.csv") 
    df = pd.DataFrame()
    for f in files: 
        csv = pd.read_csv(f)
        #appending the dataframes (Eg: Generating Dataframe 'sex' by appending male.csv and female.csv)
        df = pd.concat([df,csv]) 
    #dropping unnecessary columns
    cols_remove = [i for i in df.columns if i not in ['periodyear','period','Current','dim_title']] 
    df.drop(columns=cols_remove,inplace=True) 
    #Filtering data for 2020-2022
    ind = df[df.periodyear<=2019].index 
    df.drop(index=ind,inplace=True) 
    #Each year has an aggregated value for all months with period type 1, values at monthly level have periodtype 3. Removing aggregated values
    ind = df[df.period==0].index 
    df.drop(index=ind,inplace=True) 
    #renaming columns intuitively
    df.rename(columns={'periodyear':'year','period':'month','dim_title':str,'Current':'claims'},inplace=True) 
    
    return df

#for every sub-folder in demographics, calling the get_claims() function
files = glob.glob("data/raw/demographics/*")
names = [i[22:] for i in files]
d = {
    name : get_claims(file,file[22:]) for name, file in zip(names, files)
}

#converting dictionary of dataframes into separate dataframes with the printed names 
print('Dataframe names : ',names)
for k in d.keys():
    exec(f"{k} = d['{k}']")

#getting the claims at an industry level as well
industry = get_claims('data/raw/industry_unemployment','industry')


Dataframe names :  ['ethnicity', 'sex', 'race']


### 2. Covid Transmission Data

In [512]:
transmission = pd.read_csv('data/raw/United_States_COVID-19_County_Level_of_Community_Transmission_Historical_Changes_-_ARCHIVED.csv')
#Filtering data for the Economic Region R11000
transmission = transmission.loc[(transmission.county_name.isin(['Anoka County','Carver County','Dakota County','Hennepin County','Ramsey County','Scott County','Washington County'])) & (transmission.state_name=='Minnesota'),:] 
print(transmission.shape) #new shape
#Removing Unnecessary Columns
transmission.drop(columns=['state_name','fips_code'],inplace=True) 
transmission.isna().sum() #null values

(7000, 7)


county_name                                             0
date                                                    0
cases_per_100K_7_day_count_change                       0
percent_test_results_reported_positive_last_7_days    512
community_transmission_level                           49
dtype: int64

### 3. Loading Common Analysis data that contained Mask Policy and daily cases

In [513]:
prev_df = pd.read_csv('data/raw/prevDataset.csv')
prev_df.drop(columns='Unnamed: 0',inplace=True)
prev_df.date = pd.to_datetime(prev_df.date)
# prev_df['masks'] = prev_df['masks'].astype('category')
prev_df.tail(3)

,days,date,cases,daily_cases,slope,14_day_avg,new_cases14,slope14,masks
1008,1008,2022-10-26,912211,0.0,0.0,910921.0,394.0,0.0,0.0
1009,1009,2022-10-27,914746,2535.0,2535.0,911286.0,365.0,-29.0,0.0
1010,1010,2022-10-28,914746,0.0,-2535.0,911652.0,366.0,1.0,0.0


### 4. Loading Census Data

In [514]:
files = glob.glob('data/raw/census'+"/*.xlsx") 
a= pd.DataFrame()
for f in files :
    df= pd.read_excel(f)
    df.drop(columns=[x for x in df.columns if x.endswith('Flags')],inplace=True)
    df[['year','quarter']] = df['Unnamed: 0'].str.split(' ',expand=True)
    df.drop(columns='Unnamed: 0',inplace=True)
    df = df.loc[df.year.isin(['2020','2021','2022']),:]
    df.reset_index(inplace=True,drop=True)
    if f == files[0]:
        a = df.copy()
    else : 
        a = pd.merge(a,df,on=['year','quarter'])


In [515]:
a.drop(columns='quarter',inplace=True)
a = a.groupby('year').mean()
a.reset_index(inplace=True)
a.year=a.year.astype('int64')
a.columns = a.columns.str.lower()
a.columns = a.columns.str.replace('alone','')
a.columns = a.columns.str.strip()
a.columns = a.columns.str.replace(' ','_')
a['total_claims'] = a.male + a.female
a.rename(columns={'administrative_and_support_and_waste_management_and_remediation_services':'administrative_and_waste_services','mining,_quarrying,_and_oil_and_gas_extraction':'mining'},inplace=True)
a.drop(columns='public_administration',inplace=True)

### Changing Data Format for Consolidation

In [516]:
def pivot_df(df):
    """
    Extracts relevant columns, applies basic data cleaning and pivots dataframe to combine all unemployment claims data into a file

    Parameters
    ----------
        df : pandas DataFrame
            Dataframe to be pivoted

    Returns
    -------
        df : pandas Datafram
            Final pivoted dataframe schema with required values for analysis 

    """
    rel_cols = [i for i in df.columns if i not in ['year','month','claims']]
    df[rel_cols[0]] = df[rel_cols[0]].str.lower()
    df[rel_cols[0]] = df[rel_cols[0]].str.strip()
    df[rel_cols[0]] = df[rel_cols[0]].str.replace(' ','_')
    pivot_table = df.pivot(columns=rel_cols,values='claims',index=['year','month'])
    pivot_table.reset_index(inplace=True)
    pivot_table.columns.name=None
    return pivot_table

In [517]:
#Pivoting all unemployment claims data
pivot_sex = pivot_df(sex)
pivot_race = pivot_df(race)
pivot_ethnicity = pivot_df(ethnicity)
pivot_industry = pivot_df(industry)

In [518]:
#Cleaning up the data from the common analysis as it was performed at a daily level and now the data has been rolled up to a monthly level

#Getting relevant columns
prev_df = prev_df.loc[:,['date','cases','masks']]

#Extracting month and year
prev_df['year']=prev_df.date.dt.year
prev_df['month']=prev_df.date.dt.month

#Aggregating cases monthly by taking the sum
monthly_cases = pd.DataFrame(prev_df.groupby(['year','month'])['cases'].sum())
monthly_cases.reset_index(inplace=True)

#Rolling up mask policy to monthly level (if mask policy was in effect for a majority of the month then mask policy is in effect)
mask_policy_monthly = pd.DataFrame(prev_df.groupby(['year','month'])['masks'].max())
mask_policy_monthly.reset_index(inplace=True)
#Calculating the change in cases monthly
monthly_cases['monthly_change_cases'] = monthly_cases['cases'].sub(monthly_cases['cases'].shift())
monthly_cases['monthly_change_cases'].iloc[0]=monthly_cases['monthly_change_cases'].iloc[0]
#Calculating the change in the new cases monthly (rate of change of monthly new cases)
monthly_cases['monthly_rate_of_change_cases'] = monthly_cases['monthly_change_cases'].sub(monthly_cases['monthly_change_cases'].shift())
monthly_cases['monthly_rate_of_change_cases'].iloc[0]=monthly_cases['monthly_rate_of_change_cases'].iloc[0]

/var/folders/d6/y9k8fd5d2r36cln9ssdj6kqw0000gn/T/ipykernel_69926/4195800496.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_cases['monthly_change_cases'].iloc[0]=monthly_cases['monthly_change_cases'].iloc[0]
/var/folders/d6/y9k8fd5d2r36cln9ssdj6kqw0000gn/T/ipykernel_69926/4195800496.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_cases['monthly_rate_of_change_cases'].iloc[0]=monthly_cases['monthly_rate_of_change_cases'].iloc[0]


In [519]:
#Cleaning up Transmission Data

#Extracting date and storing month and year
transmission.date = pd.to_datetime(transmission.date)
transmission['month'] = transmission.date.dt.month
transmission['year'] = transmission.date.dt.year

#Transmission levels are provided at a daily granularity, hence I am rolling it up to the monhtly level by taking mode 
#(if most days of the month were high transmission then that month is considered as high transmission )
pivot_transmission = pd.DataFrame(transmission.groupby(['year','month'])['community_transmission_level'].agg(pd.Series.mode))
pivot_transmission.reset_index(inplace=True)

### Data Consolidation

In [520]:
#All datasets to combine
data_frames = [pivot_sex, pivot_race, pivot_ethnicity, pivot_industry,pivot_transmission,mask_policy_monthly,monthly_cases]
df_combined = reduce(lambda  left,right: pd.merge(left,right,on=['year','month'],), data_frames)
df_combined.drop(columns=['ethnicity_unknown','race_unknown','unclassified','public_administration','utilities'],inplace=True)
df_combined.rename(columns={'black_or_african_american,':'black_or_african_american','native_hawaiian/pacific_islander':'native_hawaiian_or_other_pacific_islander','other_services,_ex._public_admin':'other_services_(except_public_administration)','professional_and_technical_services':'professional,_scientific,_and_technical_services','agriculture,_forestry,_fishing_&_hunting':'agriculture,_forestry,_fishing_and_hunting'},inplace=True)

In [521]:
#Standardizing all claims data using the corresponding population of groups
no_cols = list(set(df_combined.columns)-set(a.columns))
no_cols.append('year')
no_cols
a.columns = a.columns+'_census'
a.rename(columns={'year_census':'year'},inplace=True)
df_combined = df_combined.merge(a,on='year',how='left')
for i in list(set(df_combined.columns)-set(no_cols)-set(a.columns)):
    df_combined[i] = df_combined[i]/df_combined[i+'_census']
df_combined.drop(columns=[i for i in a.columns if i != 'year'],inplace=True)

In [522]:
#Saving the consolidated dataset
df_combined.to_csv('data/consolidated/consolidatedDataset.csv',index=False)

### Schema of the final Data Frame
1. Time colums : 'year' , 'month' (2)
2. Claims Data(41)
    - sex : 'female', 'male' (2)
    - race : 'american_indian_or_alaska_native','asian', 'black_or_african_american,','native_hawaiian/pacific_islander', 'race_unknown', 'white'(8)
    - ethnicity : 'ethnicity_unknown', 'hispanic_or_latino','not_hispanic_or_latino'(3)
    - industry : 'accommodation_and_food_services',
       'administrative_and_waste_services',
       'agriculture,_forestry,_fishing_&_hunting',
       'arts,_entertainment,_and_recreation', 'construction',
       'educational_services', 'finance_and_insurance',
       'health_care_and_social_assistance', 'information',
       'management_of_companies_and_enterprises', 'manufacturing', 'mining',
       'other_services,_ex._public_admin',
       'professional_and_technical_services', 'public_administration',
       'real_estate_and_rental_and_leasing', 'retail_trade', 'total_claims',
       'transportation_and_warehousing', 'unclassified', 'utilities',
       'wholesale_trade'
        (22)
3. Community Transmission Level : 'community_transmission_level' (1)
5. mask policy data : 'masks'(1)
6. cases : 'cases', 'monthly_change_cases',
       'monthly_rate_of_change_cases' (3)